In [ ]:
"""
Create a map of all Swiss cantons with their names
"""
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon

def create_switzerland_cantons_map(output_file='output/switzerland_cantons.png'):
    """
    Create a map showing all Swiss cantons with their names
    """
    
    print("Downloading Swiss canton boundaries...")
    
    # List of all 26 Swiss cantons
    cantons = [
        'Zurich', 'Bern', 'Lucerne', 'Uri', 'Schwyz', 'Obwalden', 'Nidwalden',
        'Glarus', 'Zug', 'Fribourg', 'Solothurn', 'Basel-Stadt', 'Basel-Landschaft',
        'Schaffhausen', 'Appenzell Ausserrhoden', 'Appenzell Innerrhoden', 
        'St. Gallen', 'Grisons', 'Aargau', 'Thurgau', 'Ticino', 'Vaud', 
        'Valais', 'Neuchâtel', 'Geneva', 'Jura'
    ]
    
    # Alternative: Use a direct GeoJSON/shapefile source
    # This is more reliable than querying each canton individually
    url = "https://raw.githubusercontent.com/interactivethings/swiss-maps/master/2023/ch-cantons.json"
    
    try:
        print("Downloading from Swiss maps repository...")
        cantons_gdf = gpd.read_file(url)
        
        # Create the plot
        fig, ax = plt.subplots(figsize=(16, 12))
        
        # Plot canton boundaries
        cantons_gdf.boundary.plot(ax=ax, linewidth=1.5, edgecolor='black')
        cantons_gdf.plot(ax=ax, color='lightblue', alpha=0.5, edgecolor='black', linewidth=1.5)
        
        # Add canton names as labels at centroids
        for idx, row in cantons_gdf.iterrows():
            centroid = row.geometry.centroid
            name = row['name'] if 'name' in row else row['NAME'] if 'NAME' in row else str(idx)
            ax.text(centroid.x, centroid.y, name, fontsize=9, ha='center', va='center',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))
        
    except Exception as e:
        print(f"Could not download from repository: {e}")
        print("Using alternative method with OSMnx...")
        
        # Fallback: Download Switzerland and try to get cantons
        import osmnx as ox
        
        # Get Switzerland boundary
        switzerland = ox.geocode_to_gdf('Switzerland')
        
        fig, ax = plt.subplots(figsize=(16, 12))
        switzerland.plot(ax=ax, color='lightblue', alpha=0.5, edgecolor='black', linewidth=2)
        
        # Try to download a few major cantons as examples
        major_cantons = ['Geneva', 'Vaud', 'Zurich', 'Bern', 'Basel-Stadt']
        canton_gdfs = []
        
        for canton in major_cantons:
            try:
                gdf = ox.geocode_to_gdf(f'{canton}, Switzerland')
                gdf['name'] = canton
                canton_gdfs.append(gdf)
            except:
                print(f"Could not download {canton}")
        
        if canton_gdfs:
            all_cantons = gpd.GeoDataFrame(pd.concat(canton_gdfs, ignore_index=True))
            all_cantons.boundary.plot(ax=ax, linewidth=1.5, edgecolor='red')
            
            # Add labels
            for idx, row in all_cantons.iterrows():
                centroid = row.geometry.centroid
                ax.text(centroid.x, centroid.y, row['name'], fontsize=10, ha='center', va='center',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
    
    ax.set_title('Swiss Cantons', fontsize=18, fontweight='bold', pad=20)
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude', fontsize=12)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=150, bbox_inches='tight')
    print(f"\nSaved: {output_file}")
    plt.close()
    
    print("Done!")


# Main execution
if __name__ == "__main__":
    create_switzerland_cantons_map('figure/switzerland_cantons.png')

Could not download from repository: HTTP Error 404: Not Found
Using alternative method with OSMnx...

Saved: figure/switzerland_cantons.png
Done!
